# MIDAS INTERNSHIP PROJECT 


In [12]:
# Importing neccassary 3rd party as well in-built Python libraries
import requests
import collections
import os
import oauth2
from requests.auth import HTTPBasicAuth
import json
import pandas as pd

In [3]:
def initialise_auth_keys():
    """
    return: tuple of auth credentials to use twitter tweets API
    """
    return os.environ["TWITTER_ACCESS_TOKEN"], os.environ["TWITTER_ACCESS_TOKEN_SECRET"]

In [23]:
class MidasTweetParser:
    def __init__(self):
        self.consumer_key = os.environ["TWITTER_ACCESS_TOKEN"]
        self.consumer_secret = os.environ["TWITTER_ACCESS_TOKEN_SECRET"]
        self.request_token_url = "https://api.twitter.com/oauth2/token?grant_type=client_credentials"
        self.tweets_url = "https://api.twitter.com/1.1/statuses/user_timeline.json"
        self.tweets_url_params = {"screen_name":"midasiiitd"}
        self.tabular_data = pd.DataFrame([])
    
    def _get_bearer_token(self):
        response = requests.post(self.request_token_url, auth=HTTPBasicAuth(self.consumer_key, self.consumer_secret))
        response_data = json.loads(response.content.decode())
        return response_data["access_token"]

    def _request_and_save_tweets(self):
        bearer_token = self._get_bearer_token()
        authorization_header = self._build_bearer_token(bearer_token)
        
    def _build_bearer_token(self, token):
        return "{0} {1}".format("Bearer", token)
    
    def _get_json_for_tweets(self):
        bearer_token = self._get_bearer_token()
        authorization_header = self._build_bearer_token(bearer_token)
        response = requests.get(self.tweets_url,
                                headers={"Authorization":authorization_header}, params=self.tweets_url_params)
        return json.loads(response.content)
    
    def save_tweets(self):
        tweets_data = self._get_json_for_tweets()

In [24]:
obj = MidasTweetParser()

In [25]:
obj.tabular_data

,a
0,1


<Response [400]>

NameError: name 's' is not defined